In [ ]:
'''
+------------------+--------------------------------------------------------------+
│ Column Name      │ Description                                                  │
│------------------│--------------------------------------------------------------│
+------------------+--------------------------------------------------------------+
│ Year             | Reference Year                                               |
| Month            | Reference Month                                              │
│ GEO              │ Geographic region (encoded as Province 1, Province 2, etc.)  │
│ Products         │ Name of the retail product                                   │
│ COORDINATE       │ Internal coordinate identifier for product                   │
│ VALUE            │ Original product price per unit (before tax)                 │
│ UOM              │ Unit of measure (e.g., Dollars)                              │
│ Taxable          │ 'Yes' if the product is subject to tax, 'No' otherwise       │
│ Total tax rate   │ Total tax rate (%) based on province or average for 'Canada' │
│ Value after tax  │ Price after tax if taxable, else same as VALUE               │
│ Product Category │ High-level category assigned to the product                  │
│ Essential        │ 'Essential' for basic needs or 'Non-Essential'               │
+------------------+--------------------------------------------------------------+
'''

In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_PATH = 'data\\Retail_Prices_of _Products.csv'

In [ ]:
data= pd.read_csv(DATA_PATH)
# data

In [5]:
#반응 변수는 Value after tax으로 하고, 다른것들중에 설명변수
#각 품목을 기준으로, 시간대별 정렬. 그다음에 분석

In [ ]:
#각 변수별 기본 통계랑. 평균, 분산. 범주형의 경우, 범주별 비율



In [ ]:
#설명변수와 반응변수간의 산점도

In [ ]:
#데이터 전처리
#년도와 월을 합쳐서 시계열 데이터로 변환
da

